<a href="https://colab.research.google.com/github/dhiyasalmas/SteadyPWR/blob/main/SteadyPWR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Simulasi Thermal Hidroulik PWR**

# **Import module**

In [ ]:
import numpy as np

# **Import Constants**

In [ ]:
# Variable
def g1 ():
  global rpelet,rclin,rclout,rcool,qfuel,coflow,msr,msz

def g2 ():
  global wzrfmt,wpufmt,porfp,baeffu

def g3 ():
  global tavpel,tavcld,tavgap,tavcol

def sth2 ():
  global tpinp, tpout

def sth3():
  global maxb,gtot,psist,hcheam,graf

def sth4():
  global maxc,maxm,maxp

# **Tetapan Untuk PWR**

In [ ]:
def conw(hw, pw):
  xx = hw/5.815e+5
  conw=0.57374+0.25361*xx-0.14547*xx**2+0.013875*xx**3
  return

#Menghitung densitas air dengan masukan entalpi dan tekanan
def denw(hw, pw):
  if hw == 6.513e5:
    denw=(999.65+4.9737e-7*pw)+(-2.5847e-10+6.1767e-19*pw)*hw**2+(1.2696e-22-4.9223e-31*pw)*hw**4
  else:
    denw=(1488.64+1.3389e-6*pw)+(1.4695e9+8.85736*pw)/(hw-3.20372e6-1.20483e-2*pw)
  return

#Menghitung entalpi air dengan masukan tekanan pada kondisi saturasi
def entw(pw):
  entw=5.7474e+5+2.09206e-1*pw-2.8051e-8*pw**2+2.38098e-15*pw**3-1.0042e-22*pw**4+1.6587e-30*pw**5
  return

#menghitung entalpi air sub cooled dengan masukan tekanan  dan temperatur
def entsbw(pw, twl):
  tw=tw1+273.15
  temst=temsw(pw)
  entst=entw(pw)
  cpst=cpw(entst,pw)
  ent1=entst+(tw-temst)*cpst
  tvtst=temw(ent1,pw)
  if abs((tvtst-tw)/tw) == 0.001:
    entsbw = entl
    #disini ada write
    for i in range (1, 25):
      ent1=ent1+(tw-temst)/(tvtst-temst)*(tw-tvtst)*cpst
      tvtst=temw(ent1,pw)
      if abs((tvtst-tw)/tw) == 0.001:
        entsbw = entl
  else:
    print("entsbw do not convergent!")
  return

#Mengitung temperatur air sub cooled dengan masukan tekanan  dan entalpi
def temw(hw, pw):
  temw=(2.7291e2-1.5954e-7*pw)+(2.3949e-4-5.1963e-13*pw)*hw+(5.9660e-12+1.2064e-18*pw)*hw**2-(1.3147e-17+5.6026e-25*pw)*hw**3
  return

#menghitung cp  air sub cooled dengan masukan tekanan  dan entalpi
def cpw(hw, pw):
  a0p=2.3949e-4-5.1963e-13*pw
  a1p=1.1932e-11+2.4127e-18*pw
  a2p=-3.9441e-17-1.6808e-24*pw
  cpw=1.0/(a0p+a1p*hw+a2p*hw**2)
  return

#menghitung temperatur saturasi  air dengan masukan tekanan
def temsw(pw):
  entw1=entw(pw)
  temsw=temw(entw1,pw)
  return

#menghitung viskositas air sub cooled dengan masukan tekanan  dan entalpi
def visw(hw, pw):
  if hw == 2.76e5:
    xx=8.5813e-6*(hw-4.2659e4)
    ee=6.4845e-6*(hw-5.5359e4)
    visw=1.2995e-3-9.2640e-4*xx+3.8105e-4*xx**2-8.2194e-5*xx**3+7.0224e-6*xx**4-(-6.5959e-12+6.763e-12*ee-2.8883e-12*ee**2+4.4525e-13*ee**3)*(pw-6.8946e5)
  elif hw == 3.94e5:
    e0h=1.4526e-3-6.9881e-9*hw+1.521e-14*hw**2-1.2303e-20*hw**3
    e1h=-3.8064e-11+3.9285e-16*hw-1.2586e-21*hw**2+1.2860e-27*hw**3
    visw=e0h+e1h*(pw-6.8946e5)
  else:
    zz=3.8921e-6*(hw-4.0147e5)
    visw=3.026e-4-1.8366e-4*zz+7.5671e-5*zz**2-1.6479e-5*zz**3+1.4165e-6*zz**4
  return

# **Import Data**

In [ ]:
class inpl:
  global rpelet,rclin,rclout,rcool,qfuel,coflow,msr,msz,maxc,maxm,maxp
  from google.colab import drive
  drive.mount('/content/drive/My Drive/initherm.inp')
  #Faktor friksi laminer
  def fricpl(re):
    fricpl=16.0/re
    return
  #faktor friksi turbulen
  def fricpt(re):
    fricpt=0.0791*re**(-0.25)
    return

## **Subroutine Thermcool**

In [ ]:
# Function to calculate thermcoo1
class thermcoo1():
    #global tavpel, tavcld, tavgap, tavcol
    global xmass, tempr, press, hgt, presdp, drhead
    global tpinp, tpout, xdens, xvel, xxx, zzz, dlp, pwrb, dxx, dzz
    global tpc, denct, visct, conct, cpct, tpm, denmt, conmt, cpmt
    global rpelet, rclin, rclout, rcool, qfuel, coflow, msr, msz
    #global wzrfmt, wpufmt, porfp, baeffu, tavpel, tavcld, tavgap, tavcol
    global maxb, gtot, psist, hcheam, graf, maxc, maxm, maxp, tpc, denct, visct, conct, cpct, tpm, denmt, conmt, cpmt
    #global xdens, xvel, xxx, zzz, dlp, pwrb, dxx, dzz

    #Menghitung temperatur di seluruh jalur pendingin
    tempr[0] = tpinp
    for i in range(1, maxb-1):
        tptmp = tempr[i]
        htp = entsbw(psist, tptmp)
        cpwtp = cpw(htp, tpinp)
        tempr[i+1] = tempr[i] + pwrb(i)/gtot/cpwtp
        print('i+1:', i+1, tempr[i+1], tptmp, htp, cpwtp)

    #Mengitung pressure drop grafitasi
    drhead1=0.0
    drhead2=0.0
    hcheam=10.0
    graf=10.0

    #Bagian updtream
    for i in range(1, 14):
        tptmp = tempr(i)
        htp = entsbw(psist, tptmp)
        dentp = denw(htp, tptmp)
        drhead1 += dentp * graf * dzz(i)
        if i == 9:
            drhead1 += dentp * graf * hcheam(i)

    #Bagian downstream
    for i in range (15, 27):
      tptmp = tempr(i)
      htp = entsbw(psist,tptmp)
      dentp = denw(htp,tptmp)
      drhead2 += dentp * graf * dzz(i)
      if i == 23:
        drhead2 += dentp * graf * hcheam

NameError: name 'tpinp' is not defined

# **Subroutine prdrp2**

In [ ]:
class prdrp2:
   global tpinp,tpout,maxb,gtot,psist,hcheam,graf,maxc,maxm,maxp
   #Mengitung prressure drop akibat friksi
   power=3.0E8
   vol=3.0
   area=2.0
   crat=0.35
   tptmp=tempr[4]
   print("tptmp :", tptmp)
   htp=entsbw(psist,tptmp)
   print('htp:',htp)
   cptp=cpw(htp,tptmp)
   print('cptp:',cptp)
   dentp=denw(htp,tptmp)
   print('dentp:',dentp)
   flowt=power/cptp/40.0/area/crat
   print('flowt:',flowt)
   drhead1=dentp*graf*dzz(4)
   p=0.0125
   d=0.010
   s=0.001
   print('tptmp,htp,cptp,dentp,flowt,drhead1')
   print(tptmp,htp,cptp,dentp,flowt,drhead1)
   print('j,htp,cptp,de,denw1,visw1,re,fsm,pdrop')
   pdrop=0.0
   flowt1=flowt*crat*area
   dpipe=0.4
   apipe=3.14159*dpipe**2/4.0

   #Sebelum teras reaktor
   print("j,de,velo,re,fsm,dpdrop,pdrop")
   for j in range (1, 3):
    tptmp = tempr(j)
    htp=entsbw(psist,tptmp)
    cptp=cpw(htp,tptmp)
    de=dpipe
    denw1=denw(htp,psist)
    visw1=visw(htp,psist)
    velo=flowt1/denw1/apipe
    re=denw1*velo*de/visw1
    fsm=fricpt(re)
    dpdrop=velo**2/(2.0*de)*fsm*denw1*(dxx(j)+dzz(j))
    pdrop += dpdrop
    print(j,de,velo,re,fsm,dpdrop,pdrop)
    continue

    rco=p/(3.14159)**0.5
    areaco=3.14159*(rco**2-d**2/4.0)
    perim=2*3.14159*(rco+d/2)
    de=4*areaco/perim

    #Bagian teras reaktor
    print('pitch rco de:',p,rco,de)
    for j in range (4, 7):
      tptmp=tempr(j)
      htp=entsbw(psist,tptmp)
      cptp=cpw(htp,tptmp)
      denw1=denw(htp,psist)
      visw1=visw(htp,psist)
      velo=flowt/denw1
      re=denw1*velo*de/visw1
      fsm=fricpt(re)
      dpdrop=velo**2/(2.0*de)*fsm*denw1*dzz(j)
      pdrop+=dpdrop
      print(j,de,velo,re,fsm,dpdrop,pdrop)
      continue

    #Antara teras dengan steam generator
    for j in range (8, 17):
       tptmp=tempr(j)
       htp=entsbw(psist,tptmp)
       cptp=cpw(htp,tptmp)
       de=dpipe
       denw1=denw(htp,psist)
       visw1=visw(htp,psist)
       velo=flowt1/denw1/apipe
       re=denw1*velo*de/visw1
       fsm=fricpt(re)
       dpdrop=velo**2/(2.0*de)*fsm*denw1*(dxx(j)+dzz(j))
       pdrop=pdrop+dpdrop
       print(j,de,velo,re,fsm,dpdrop,pdrop)
       continue

       rsgi=0.5
       rsgo=1.0
       areaco=3.14159*(rsgo**2-rsgi**2)
       perim=2*3.14159*(rsgo+rsgi)
       de=4*areaco/perim
       flowt2=flowt/3

       #Antara steam generator dengan pompa
       print('pitch rco de:',p,rco,de)
       for j in range (18, 21):
          tptmp=tempr(j)
          htp=entsbw(psist,tptmp)
          cptp=cpw(htp,tptmp)
          denw1=denw(htp,psist)
          visw1=visw(htp,psist)
          velo=flowt2/denw1
          re=denw1*velo*de/visw1
          fsm=fricpt(re)
          dpdrop=velo**2/(2.0*de)*fsm*denw1*abs(dzz(j))
          pdrop=pdrop+dpdrop
          print(j,de,velo,re,fsm,dpdrop,pdrop)
          continue

# **Subroutine initl**

In [ ]:
class initl:
  power=3.0E8
  tpinp=300.0
  tpout=340.0
  hinlet=entsbw(psist,tpinp)
  houtl=entsbw(psist,tpout)
  cpwin=cpw(hinlet,tpinp)
  cpwout=cpw(houtl,tpout)
  gtot=power/(tpout-tpinp)/(cpwin+cpwout)*2.0
  print("hinlet, houtlet :", hinlet, houtl)
  print('cpinlet, cpoutlet :', cpwin, cpwout)
  maxb=28
  psist=2.0E7
  hsat=entw(psist)
  tpfu1=800.0
  print('h sat:', hsat)

SyntaxError: cannot assign to function call here. Maybe you meant '==' instead of '='? (<ipython-input-8-bbd31f39d941>, line 18)